In [62]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
from matplotlib import pyplot as plt
import json 
import ast
import os
%pylab inline


Populating the interactive namespace from numpy and matplotlib


In [94]:
file_list = [e for e in os.listdir('.') if e[-3:] == 'csv']
#print file_list
projects = {}
for f in file_list:
    projects[f] = pd.read_csv(f, sep='\t', index_col=0)
projects['fiverrUsersPic_analyze_with_projectposition_ALL.csv'] = pd.concat(projects.values())
d = projects["fiverrUsersPic_analyze_with_projectposition_ALL.csv"]

In [95]:
d.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 88003 entries, 0 to 6415
Data columns (total 32 columns):
user_id                           88003 non-null object
gender                            88003 non-null object
gender_confidence                 88003 non-null float64
race                              88003 non-null object
race_confidence                   88003 non-null float64
age_value                         88003 non-null int64
age_range                         88003 non-null int64
smiling                           88003 non-null float64
category                          88003 non-null object
position                          88003 non-null object
url                               40736 non-null object
gig_categories                    14792 non-null object
languages                         86598 non-null object
overall_rating                    88003 non-null float64
num_reviews                       88003 non-null int64
num_projects                      88003 non-null in

In [96]:
# this is what the dictionaries look like
d[pd.notnull(d.position_on_screen_high_rating)][:5]['position_on_screen_high_rating']

0    {692562: 156}
1               {}
2               {}
3               {}
4               {}
Name: position_on_screen_high_rating, dtype: object

In [97]:
# For every user we calculate the average position of their tasks in the file
def process_dict(dict_str):
    d = ast.literal_eval(dict_str)
    if d != {}:
        a = [d[k] for k in d.keys() if d[k] != None]
        if a == []: 
            avg = 0
        else: 
            avg = float(sum(a))#/float(len(a))
    else:
        avg = -1
    return avg

In [98]:
# We create new variables with this average as 3 more columns
for d in projects.values():
    d['position_on_screen_high_rating_avg'] = d.position_on_screen_high_rating.apply(process_dict)
    d['position_on_screen_recommended_avg'] = d.position_on_screen_recommended.apply(process_dict)
    d['position_on_screen_new_avg'] = d.position_on_screen_new.apply(process_dict)


In [99]:
d.icol(slice(29,35))[:5].T

,0,1,2,3,4
position_on_screen_high_rating,{11890143: None},{},{},{},{}
position_on_screen_recommended,{11890143: 83},{},{},{},{}
position_on_screen_new,{},{},{},{},{}
position_on_screen_high_rating_avg,0,-1,-1,-1,-1
position_on_screen_recommended_avg,83,-1,-1,-1,-1
position_on_screen_new_avg,-1,-1,-1,-1,-1


In [103]:
#number of tasks
print "Number of tasks with a non zero position  a-high-rating, b-recommended, c-new :"
for k in sorted(projects.keys()):
    d = projects[k]
    print k[44:-4] + "\n a," + str(len(d.position_on_screen_high_rating_avg[d.position_on_screen_high_rating_avg > 0])) + " b, " + str(len(d.position_on_screen_recommended_avg[d.position_on_screen_recommended_avg > 0]))+ " c, " + str(len(d.position_on_screen_new_avg[d.position_on_screen_new_avg > 0]))
    print
    

Number of tasks with a non zero position  a-high-rating, b-recommended, c-new :
ALL
 a,3493 b, 7743 c, 2043

Animation_and_3D
 a,699 b, 948 c, 117

Banner_Advertising
 a,430 b, 1065 c, 184

Databases
 a,268 b, 780 c, 223

Diet_and_Weight_Loss
 a,109 b, 551 c, 169

Financial_Consulting
 a,151 b, 470 c, 128

Singers_and_Songwriters
 a,465 b, 1257 c, 214

T_Shirts
 a,427 b, 1497 c, 567

Translation
 a,736 b, 456 c, 252

Web_Analytics
 a,208 b, 719 c, 189



In [104]:
print "correlation between POSITION and RATING, (high-rating, recommended, new)"
for k in sorted(projects.keys()):
    d = projects[k]
    pos_list = [d.position_on_screen_high_rating_avg, d.position_on_screen_recommended_avg, d.position_on_screen_new_avg]
    print k[44:-4]
    for e in pos_list:
        print e[e > 0].corr(d.overall_rating)
    print


correlation between POSITION and RATING, (high-rating, recommended, new)
ALL
0.117865318528
0.0424337287143
0.070611110566

Animation_and_3D
0.0453647787949
0.0848167988259
-0.0261783062343

Banner_Advertising
0.134530844072
5.7224676293e-05
0.302691087421

Databases
0.0631804668433
0.164631094697
0.124584863946

Diet_and_Weight_Loss
0.00687162275802
0.0317700931901
0.0749289925078

Financial_Consulting
0.0962134152584
-0.0485078711263
0.157499549429

Singers_and_Songwriters
0.0238129327092
-0.278819293495
0.25929981863

T_Shirts
-0.000596148961235
-0.00603690558856
0.206271043274

Translation
-0.0172112799338
0.148254619565
0.0428959331313

Web_Analytics
0.075633245774
-0.00783965341657
0.174068624035



In [105]:
print "correlation between POSITION and NUMBER OF REVIEWS, (high-rating, recommended, new)"
for k in sorted(projects.keys()):
    d = projects[k]
    pos_list = [d.position_on_screen_high_rating_avg, d.position_on_screen_recommended_avg, d.position_on_screen_new_avg]
    print k[44:-4]
    for e in pos_list:
        print e[e > 0].corr(d.num_reviews) 
    print

correlation between POSITION and NUMBER OF REVIEWS, (high-rating, recommended, new)
ALL
0.00407953036118
0.0258231581949
0.0169199296778

Animation_and_3D
-0.108719886835
0.03041229118
0.0419434556467

Banner_Advertising
0.123565898012
0.202072610186
0.132094916784

Databases
0.130534087177
0.120058305386
0.166445046343

Diet_and_Weight_Loss
0.0344198262332
0.064088231903
0.112590926644

Financial_Consulting
-0.0563898283073
0.00528430692162
0.144318012014

Singers_and_Songwriters
-0.0776305579296
-0.0457471486145
0.0736008319191

T_Shirts
-0.0757078284899
0.0204768451141
-0.00898623921812

Translation
-0.235346104857
0.29349474923
0.0688371925201

Web_Analytics
-0.0480066747051
-0.0150534425636
-0.113058377312



In [107]:
d.gender[d.position == 'seller'].value_counts()

-         396
Male      210
Female    101
dtype: int64

In [117]:
print "position of FEMALES vs MALES (vs -)"
for k in sorted(projects.keys()):
    d = projects[k]
    pos_list = [d.position_on_screen_high_rating_avg, d.position_on_screen_recommended_avg, d.position_on_screen_new_avg]
    print k[44:-4]
    for e in pos_list:
        print np.mean(e[np.logical_and((d.gender == "Female"),(e !=-1 ))]), np.mean(e[np.logical_and((d.gender == "Male"),(e !=-1 ))]), np.mean(e[np.logical_and((d.gender == "-"),(e !=-1 ))])
    print

position of FEMALES vs MALES (vs -)
ALL
289.382352941 260.088235294 290.310179641
692.665634675 674.987772926 747.468315617
238.132315522 178.384615385 217.926356589

Animation_and_3D
324.532608696 299.414110429 377.267507003
722.631578947 692.054404145 746.844139651
152.6 100.04 117.766666667

Banner_Advertising
341.568181818 230.58974359 246.84423676
731.147928994 661.328125 696.045994065
82.8095238095 115.321428571 108.634146341

Databases
170.857142857 144.841584158 162.661870504
527.578947368 500.631746032 554.099273608
118.176470588 123.213592233 123.269230769

Diet_and_Weight_Loss
111.391304348 91.725 111.611111111
361.535087719 371.076923077 418.178343949
99.962962963 72.0 84.7173913043

Financial_Consulting
102.028571429 78.402173913 96.1857142857
433.566666667 344.527607362 438.315625
120.366666667 89.3571428571 116.952380952

Singers_and_Songwriters
275.204081633 297.369047619 314.451178451
792.465608466 829.420245399 933.495460441
142.4 120.137931034 122.877862595

T_Shirts

In [110]:
d.race[d.position == 'seller'].value_counts()

-        396
White    251
Asian     38
Black     22
dtype: int64

In [116]:
print "position of ASIANS, BLACKS and WHITES"
for k in sorted(projects.keys()):
    d = projects[k]
    pos_list = [d.position_on_screen_high_rating_avg, d.position_on_screen_recommended_avg, d.position_on_screen_new_avg]
    print k[44:-4]
    for e in pos_list:
        print np.mean(e[np.logical_and((d.race == "Asian"),(e !=-1 ))]), np.mean(e[np.logical_and((d.race == "Black"),(e !=-1 ))]), np.mean(e[np.logical_and((d.race == "White"),(e !=-1 ))]), np.mean(e[np.logical_and((d.race == "-"),(e !=-1 ))])
    print    


position of ASIANS, BLACKS and WHITES
ALL
307.170212766 228.629807692 269.407476636 290.310179641
717.111954459 702.904761905 671.03501506 747.468315617
253.481751825 166.247863248 197.215206186 217.926356589

Animation_and_3D
358.720430108 274.675675676 299.471052632 377.267507003
815.673469388 799.980392157 666.878923767 746.844139651
139.727272727 160.090909091 105.819277108 117.766666667

Banner_Advertising
244.290322581 214.2 290.886792453 246.84423676
641.358208955 668.822222222 702.223642173 696.045994065
109.3 134.555555556 101.603448276 108.634146341

Databases
149.8125 117.882352941 157.359223301 162.661870504
475.222222222 473.619047619 514.014035088 554.099273608
118.454545455 134.647058824 120.739130435 123.269230769

Diet_and_Weight_Loss
47.5 37.8 110.711538462 111.611111111
302.666666667 332.551724138 378.634020619 418.178343949
84.9090909091 87.5555555556 92.4666666667 84.7173913043

Financial_Consulting
184.166666667 39.2727272727 82.9032258065 96.1857142857
372.928571